In [1]:
import sys
import json
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
current_dir = Path.cwd()
project_root = current_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print('done')

done


In [3]:
from src.utils.save_data import save_df_parquet_safe, load_df_parquet_safe

In [4]:
# loading the master_df
path = project_root / 'data' / 'processed' / 'merge' / 'aligned_avw_merged.parquet'
avw_merged = load_df_parquet_safe(path=str(path))
avw_merged.head()

,Time,words,text_concat,h_ratio,v_ratio,_neutral,browDownLeft,browDownRight,browInnerUp,browOuterUpLeft,...,mouthStretchRight,mouthUpperUpLeft,mouthUpperUpRight,noseSneerLeft,noseSneerRight,audio_rms(volumn),audio_pitch_avg,audio_pitch_var(expressiveness),is_silent,speaker
0,0.0,"[We're, starting]",We're starting,0.463844,0.012997,3.112116e-06,0.015592,0.025971,0.023290,0.123947,...,0.000443,0.000025,0.000022,4.540787e-07,4.185384e-07,0.0373,0.00,0.00,False,None
1,0.5,"[now., [*]]",now. [*],0.418074,-0.048988,2.558370e-06,0.017408,0.037184,0.007811,0.137800,...,0.000918,0.000023,0.000030,2.511999e-07,3.643879e-07,0.0533,169.25,11.45,False,A
2,1.0,None,,0.465044,-0.030748,3.958608e-07,0.008864,0.010406,0.016732,0.179330,...,0.000332,0.000189,0.000269,2.899701e-06,1.134860e-07,0.0006,0.00,0.00,True,A
3,1.5,"[So, welcome]",So welcome,0.505119,-0.055033,3.260218e-07,0.012983,0.016629,0.008106,0.186212,...,0.000170,0.000070,0.000103,2.350217e-06,9.529118e-08,0.0593,179.94,14.03,False,A
4,2.0,"[to, the]",to the,0.590369,-0.050514,5.068692e-07,0.016363,0.028436,0.006705,0.117622,...,0.001666,0.000455,0.000476,4.304491e-06,2.317813e-07,0.0646,237.84,56.17,False,A


In [5]:
type(avw_merged.loc[avw_merged['speaker'] == 'B', 'words'])

pandas.core.series.Series

In [6]:
type(avw_merged['words'][0])

list

In [7]:
# total no of words
totalwords = 0
for words in avw_merged.loc[avw_merged['speaker'] == 'B', 'words']:
    if words and words != None:
        totalwords += len(words)
    else:
        continue

In [8]:
# let's count the filler words
cnt = 0
for words in avw_merged.loc[avw_merged['speaker'] == 'B', 'words']:
    if words and words != None:
        for word in words:
            if word == '[*]':
                cnt += 1
    else:
        continue

In [9]:
print(f"Total no of words used by the user: {totalwords}")

Total no of words used by the user: 1552


In [10]:
print(f"Total no of filler words by user: {cnt}")

Total no of filler words by user: 209


In [ ]:
print(f"percentage of filler words: {(cnt/totalwords)*100}%")

percentage of filler words: 13.466494845360824%
